In [1]:
from torchvision.transforms import transforms
from torch.utils.data import Dataset
import pickle
import os
import torch
import numpy as np

#set device
device = torch.device("mps" if torch.cuda.is_available() else "cpu")

#constants
VIDEO = 40
PARTICIPANT = 32
CHANNEL = 32
SESSION = 63


#Make Datasets
class Arousal_Dataset(Dataset):
    def __init__(self, data_path, train = None):
        super().__init__
        self.data_path = data_path
        self.palette_name_list = os.listdir(data_path)

    def __getitem__(self, index):
        palette_path = os.path.join(self.data_path, self.palette_name_list[index])
        palette = pickle.load(open(palette_path, 'rb'))
        palette = np.float32(palette)
        to_tensor = transforms.ToTensor()
        transformed_palette = to_tensor(palette)
        x_data = transformed_palette

        palette_split_list = self.palette_name_list[index].split('_')
        Truth_label = palette_split_list[6]

        if Truth_label[2:4] == 'HV':
            y_data = 1
        else:
            y_data = 0

        return x_data, y_data

    def __len__(self):
        return len(self.palette_name_list)

# #데이터 전처리
# transform = transforms.Compose(
#     [transforms.ToTensor(),transforms.Normalize((0.5 , 0.5, 0.5), (0.5, 0.5, 0.5))]
# )


#총 데이터 num :32 * 40 * 63, batch_size는 이의 약수
batch_size = 8#이미지가 9 x 9 -> 사실상 81 배치
Data_Path_Train = "./Data/data_palettes_train/"
Data_Path_Test = "./Data/data_palettes_test/"


#test_데이터, train_데이터 불러오고 저장
train_set = Arousal_Dataset(data_path = Data_Path_Train, train = True)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers = 0)

test_set = Arousal_Dataset(data_path = Data_Path_Test, train = False)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle = False, num_workers = 0)

#output clasees
classes =  (0, 1)


In [11]:
import torch.nn as nn
import torch.nn.functional as F

class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(243, 81)
        self.fc2 = nn.Linear(81, 27)
        self.fc3 = nn.Linear(27, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # print(f"initial {x.shape}")
        x = self.sigmoid(x)
        # print(f"sigmoid {x.shape}")
        x = x.view(8, -1)
        # print(x.shape)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        # print(f"linear {x.shape}")
        return x

net = BinaryClassifier()


#"net" model runs through GPU
net.to(device)

#Set Optimizer and loss function
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum = 0.9)

In [12]:
from tqdm import tqdm
from tqdm import notebook
#학습
start_flag = 1
for epoch in range(2):   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in enumerate(notebook.tqdm(train_loader), 0):

        if start_flag == 1:
            print("start learning!!!")
            start_flag = 0

        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data

        #검증
        # print(f"minibatch number : {i}, file's shape : {inputs.shape}")

        #move to GPU to calculate
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

  0%|          | 0/7560 [00:00<?, ?it/s]

start learning!!!
[1,  1000] loss: 0.346
[1,  2000] loss: 0.345
[1,  3000] loss: 0.344
[1,  4000] loss: 0.344
[1,  5000] loss: 0.345
[1,  6000] loss: 0.345
[1,  7000] loss: 0.344


  0%|          | 0/7560 [00:00<?, ?it/s]

[2,  1000] loss: 0.344
[2,  2000] loss: 0.344
[2,  3000] loss: 0.344
[2,  4000] loss: 0.345
[2,  5000] loss: 0.344
[2,  6000] loss: 0.344
[2,  7000] loss: 0.344


  0%|          | 0/7560 [00:00<?, ?it/s]

[3,  1000] loss: 0.344
[3,  2000] loss: 0.344
[3,  3000] loss: 0.344
[3,  4000] loss: 0.344
[3,  5000] loss: 0.345
[3,  6000] loss: 0.345
[3,  7000] loss: 0.344


  0%|          | 0/7560 [00:00<?, ?it/s]

[4,  1000] loss: 0.345
[4,  2000] loss: 0.344
[4,  3000] loss: 0.345
[4,  4000] loss: 0.344
[4,  5000] loss: 0.343
[4,  6000] loss: 0.344
[4,  7000] loss: 0.344


  0%|          | 0/7560 [00:00<?, ?it/s]

[5,  1000] loss: 0.345
[5,  2000] loss: 0.343
[5,  3000] loss: 0.345
[5,  4000] loss: 0.344
[5,  5000] loss: 0.345
[5,  6000] loss: 0.345
[5,  7000] loss: 0.344
Finished Training


In [ ]:
#For Colab
###Save Model
# PATH = "/content/drive/MyDrive/models/"
# torch.save(net.state_dict(), PATH)

In [14]:
#Accuracy Check
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = net(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total : .2f} %')

Accuracy of the network on the 10000 test images:  61.875000 %
